In [1]:
!python -m pip install "../."

Processing c:\users\danii\program\python\project\imgdoc
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for imgdoc: filename=imgdoc-0.0.2-py3-none-any.whl size=9144 sha256=99e2ff5cf047caa171b55d704aa010eede4bcb56149c9ed784ce790e2b448b3b
  Stored in directory: C:\Users\danii\AppData\Local\Temp\pip-ephem-wheel-cache-xdb1b_35\wheels\b8\e3\4a\f1f494124c2259921bd58533df69bb605814dcc87a7223eb4d
Successfully built imgdoc
  Attempting uninstall: imgdoc
    Found existing installation: imgdoc 0.0.2
    Uninstalling imgdoc-0.0.2:
      Successfully uninstalled imgdoc-0.0.2


In [2]:
import pytesseract
from img_doc.extractors.word_extractors import BaseWordExtractor
from img_doc.data_structures import Word
import numpy as np
from typing import List

class TesseractWordExtractor(BaseWordExtractor):
    def extract_from_img(self, img: np) -> List[Word]:
        tesseract_bboxes = pytesseract.image_to_data(
            config="-l rus",
            image=img,
            output_type=pytesseract.Output.DICT)
        word_list = []
        for index_bbox, level in enumerate(tesseract_bboxes["level"]):
            if level == 5:
                word = Word(text = tesseract_bboxes["text"][index_bbox])
                word.set_point_and_size({
                    "x_top_left":tesseract_bboxes["left"][index_bbox],
                    "y_top_left":tesseract_bboxes["top"][index_bbox],
                    "width":tesseract_bboxes["width"][index_bbox],
                    "height": tesseract_bboxes["height"][index_bbox],
                })
                word_list.append(word)
        return word_list

In [3]:
from img_doc.data_structures import Image
import os

image = Image()
image.set_img_from_path(os.path.join("img", "1.jpg"))

word_ext = TesseractWordExtractor()
words = word_ext.extract_from_img(image.img)

In [4]:
from img_doc.extractors.block_extractors.block_extractor_from_word import KMeanBlockExtractor
kmeanext = KMeanBlockExtractor()


In [5]:
kmeanext.get_index_neighbors_word(words, 3)

[[0, 1, 4, 0],
 [1, 5, 4, 0],
 [2, 6, 4, 1],
 [3, 25, 7, 21],
 [0, 5, 15, 4],
 [0, 29, 16, 1],
 [6, 32, 20, 2],
 [3, 8, 23, 6],
 [3, 9, 24, 32],
 [3, 10, 24, 8],
 [10, 11, 26, 9],
 [11, 12, 26, 10],
 [12, 13, 26, 11],
 [13, 14, 36, 12],
 [14, 14, 36, 13],
 [4, 16, 18, 15],
 [5, 17, 18, 15],
 [2, 30, 19, 16],
 [16, 5, 27, 18],
 [16, 50, 28, 18],
 [16, 21, 29, 39],
 [17, 44, 30, 20],
 [6, 23, 32, 43],
 [8, 24, 33, 22],
 [8, 25, 60, 23],
 [9, 26, 34, 24],
 [12, 36, 35, 25],
 [18, 5, 38, 27],
 [19, 29, 39, 18],
 [20, 30, 40, 5],
 [21, 31, 42, 51],
 [21, 32, 42, 30],
 [22, 8, 44, 6],
 [23, 24, 46, 45],
 [25, 35, 60, 33],
 [26, 36, 60, 34],
 [13, 37, 60, 26],
 [13, 37, 62, 36],
 [27, 39, 49, 38],
 [28, 20, 50, 38],
 [29, 41, 50, 39],
 [30, 42, 50, 40],
 [30, 43, 50, 41],
 [32, 44, 52, 42],
 [32, 45, 52, 43],
 [33, 46, 60, 44],
 [33, 47, 60, 45],
 [33, 24, 60, 46],
 [38, 19, 55, 48],
 [38, 50, 55, 18],
 [40, 51, 55, 19],
 [40, 30, 55, 50],
 [40, 44, 56, 50],
 [49, 54, 72, 53],
 [49, 55, 72, 5